In [1]:

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

tf.reset_default_graph() 

np.random.seed(2019)
tf.set_random_seed(2019)

# load data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# define first layer
num_filters1 = 32

x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1])

W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,num_filters1],
                                          stddev=0.1))
h_conv1 = tf.nn.conv2d(x_image, W_conv1,
                       strides=[1,1,1,1], padding='SAME')

b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]))
h_conv1_cutoff = tf.nn.relu(h_conv1 + b_conv1)

h_pool1 = tf.nn.max_pool(h_conv1_cutoff, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')

# define second layer
num_filters2 = 64

W_conv2 = tf.Variable(
            tf.truncated_normal([5,5,num_filters1,num_filters2],
                                stddev=0.1))
h_conv2 = tf.nn.conv2d(h_pool1, W_conv2,
                       strides=[1,1,1,1], padding='SAME')

b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]))
h_conv2_cutoff = tf.nn.relu(h_conv2 + b_conv2)

h_pool2 = tf.nn.max_pool(h_conv2_cutoff, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')

# define fully connected layer
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*num_filters2])

num_units1 = 7*7*num_filters2
num_units2 = 1024

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b2 = tf.Variable(tf.constant(0.1, shape=[num_units2]))
hidden2 = tf.nn.relu(tf.matmul(h_pool2_flat, w2) + b2)

keep_prob = tf.placeholder(tf.float32)
hidden2_drop = tf.nn.dropout(hidden2, keep_prob)

w0 = tf.Variable(tf.zeros([num_units2, 10]))
b0 = tf.Variable(tf.zeros([10]))
k = tf.matmul(hidden2_drop, w0) + b0
p = tf.nn.softmax(k)

#define loss (cost) function
t = tf.placeholder(tf.float32, [None, 10])
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=k,labels=t)) 
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# prepare session
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

# start training
i = 0
for i in range(501):
    batch_xs, batch_ts = mnist.train.next_batch(50)
    sess.run(train_step,
             feed_dict={x:batch_xs, t:batch_ts, keep_prob:0.5})
    if i % 100 == 0:
        loss_vals, acc_vals = [], []
        for c in range(4):
            start = int(len(mnist.test.labels) / 4 * c)
            end = int(len(mnist.test.labels) / 4 * (c+1))
            loss_val, acc_val = sess.run([loss, accuracy],
                feed_dict={x:mnist.test.images[start:end],
                           t:mnist.test.labels[start:end],
                           keep_prob:1.0})
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
        loss_val = np.sum(loss_vals)
        acc_val = np.mean(acc_vals)
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))

C:\Users\B-7\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on 

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

t=Tokenizer() #토큰화, 단어에 대한 정수 인코딩
#      1    2    3  4    5      6     7
text="The earth is an awesome place live"
t.fit_on_texts([text])
test="an great place live The earth is"
t.texts_to_sequences([test])

#pad_sequences : 단어의 수를 동일한 길이로...
pad_sequences([[1,2,3], [3,4,5,6,7,8], [7]], maxlen=3)


array([[1, 2, 3],
       [6, 7, 8],
       [0, 0, 7]])

In [10]:
# 워드임베딩: 텍스트 -> 숫자(Dense Vector)
# 원핫벡터 -> [0 0 0 0 1...0] -> 고차원 -> 임베딩벡터(밀집 벡터) 0 또는 1
#              나 너 학교 구름    서울 (단어가 1000개 -> 1000차원)
# ex) 학교 -> [0  0   1    0   ... 0 ]

# 밀집벡터에서 '학교' -> [0.1 -1.5 0.8 0.9] 모든 수가 다 가능
# Embedding() : 밀집벡터 생성

text = [['Hope', 'to', 'see', 'you', 'soon'], #토큰화된 상태
        ['Nice', 'to', 'see', 'you', 'again']]
text = [[0,1,2,3,4], [5,1,2,3,6]] #정수 인코딩
# => 원핫벡터로 변환되면...
# => [[[10000000], [01000000], .... ], [[0000010]]]

# 정수인코딩 => 밀집 벡터 생성
from keras.layers import Embedding
print(Embedding(7,2,input_length=5))
#밀집벡터로 변환된다면...
# => [[1.2, 3.1], [0.1, 3.7], ... ]




In [13]:
# Sequential(): 모델 생성(입력층, 은닉층, 출력층)함수
from keras.layers import Dense
from keras.models import Sequential
model=Sequential()
model.add(Dense(1,input_dim=3, init='uniform', activation='relu'))
#Dense(출력 뉴런의수, 입력 뉴런의 수, init: 초기화(uniform:균등, normal:가우시안분포)

C:\Users\B-7\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, input_dim=3, activation="relu", kernel_initializer="uniform")`
  """


In [14]:
model.add(Dense(8,input_dim=4, init='uniform', activation='relu'))
# 입력 4, 출력 8 => 4개를 입력받아 8개를 출력하는 모델 정의\
model.add(Dense(1, activation='sigmoid'))



C:\Users\B-7\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, input_dim=4, activation="relu", kernel_initializer="uniform")`
  """Entry point for launching an IPython kernel.
